In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [2]:
# Store the CSV you created in part one into a DataFrame
city_data_df = pd.read_csv("Data/WeatherPy_challenge.csv")

In [3]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = [max(temp,0) for temp in max_temp]

In [5]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))
rain_prompt = str(input("Do you want it to be raining? (yes/no)"))
snow_prompt = str(input("Do you want it to be snowing? (yes/no)"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?95
Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)no


In [6]:
# Filter the dataset to find the cities that fit the criteria

if rain_prompt == 'yes' and snow_prompt == 'no':
    
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Rain inches (last 3 hours)"] > 0)]
    
elif rain_prompt == 'no' and snow_prompt == 'yes':
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Snow inches (last 3 hours)"] > 0)]

elif rain_prompt == 'yes' and snow_prompt == 'yes':
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Snow inches (last 3 hours)"] > 0) &
                                       (city_data_df["Rain inches (last 3 hours)"] > 0)]
        
elif rain_prompt == 'no' and snow_prompt == 'no':
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Snow inches (last 3 hours)"] == 0) &
                                       (city_data_df["Rain inches (last 3 hours)"] == 0)]
    
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
1,1,Georgetown,MY,2020-02-27 02:32:14,5.41,100.34,86.00,66,20,4.70,few clouds,0.0,0.0
2,2,Kapaa,US,2020-02-27 02:35:31,22.08,-159.32,80.60,65,20,19.46,few clouds,0.0,0.0
4,4,Hilo,US,2020-02-27 02:33:41,19.73,-155.09,80.60,51,1,11.41,clear sky,0.0,0.0
6,6,Lorengau,PG,2020-02-27 02:35:32,-2.02,147.27,85.64,69,0,4.97,clear sky,0.0,0.0
7,7,Rikitea,PF,2020-02-27 02:35:32,-23.12,-134.97,80.44,75,100,8.32,overcast clouds,0.0,0.0
11,11,Butaritari,KI,2020-02-27 02:35:33,3.07,172.79,84.34,79,100,16.53,overcast clouds,0.0,0.0
22,22,Avarua,CK,2020-02-27 02:35:35,-21.21,-159.78,78.80,94,0,3.36,light rain,0.0,0.0
26,26,Kudahuvadhoo,MV,2020-02-27 02:35:36,2.67,72.89,82.11,82,92,14.47,overcast clouds,0.0,0.0
30,30,Cartagena,CO,2020-02-27 02:32:24,10.40,-75.51,78.80,88,20,8.05,few clouds,0.0,0.0
37,37,Muyuka,CM,2020-02-27 02:35:38,4.29,9.41,78.80,88,75,3.36,broken clouds,0.0,0.0


In [7]:
preferred_cities_df.count()

City_ID                       135
City                          135
Country                       135
Date                          135
Lat                           135
Lng                           135
Max Temp                      135
Humidity                      135
Cloudiness                    135
Wind Speed                    135
Current Description           135
Rain inches (last 3 hours)    135
Snow inches (last 3 hours)    135
dtype: int64

In [8]:
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                       135
City                          135
Country                       135
Date                          135
Lat                           135
Lng                           135
Max Temp                      135
Humidity                      135
Cloudiness                    135
Wind Speed                    135
Current Description           135
Rain inches (last 3 hours)    135
Snow inches (last 3 hours)    135
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,86.00,few clouds,5.41,100.34,
2,Kapaa,US,80.60,few clouds,22.08,-159.32,
4,Hilo,US,80.60,clear sky,19.73,-155.09,
6,Lorengau,PG,85.64,clear sky,-2.02,147.27,
7,Rikitea,PF,80.44,overcast clouds,-23.12,-134.97,
11,Butaritari,KI,84.34,overcast clouds,3.07,172.79,
22,Avarua,CK,78.80,light rain,-21.21,-159.78,
26,Kudahuvadhoo,MV,82.11,overcast clouds,2.67,72.89,
30,Cartagena,CO,78.80,few clouds,10.40,-75.51,
37,Muyuka,CM,78.80,broken clouds,4.29,9.41,


In [14]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search terms: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,86.00,few clouds,5.41,100.34,Cititel Penang
2,Kapaa,US,80.60,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
4,Hilo,US,80.60,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
6,Lorengau,PG,85.64,clear sky,-2.02,147.27,Lorengau Harbourside Hotel
7,Rikitea,PF,80.44,overcast clouds,-23.12,-134.97,Pension Maro'i
11,Butaritari,KI,84.34,overcast clouds,3.07,172.79,Isles Sunset Lodge
22,Avarua,CK,78.80,light rain,-21.21,-159.78,Paradise Inn
26,Kudahuvadhoo,MV,82.11,overcast clouds,2.67,72.89,Kudahuvadhoo Retreat
30,Cartagena,CO,78.80,few clouds,10.40,-75.51,Costa Del Sol
37,Muyuka,CM,78.80,broken clouds,4.29,9.41,Muyuka Motor Park (El Paso)


In [11]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description</dt><dd>{Current Description} with temperature of {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [12]:
# Add a map for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create the output file (CSV)
output_data_file = "Data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV
hotel_df.to_csv(output_data_file, index_label="City_ID")